# Mandatory Challenge
## Context
You work in the data analysis team of a very important company. On Monday, the company shares some good news with you: you just got hired by a major retail company! So, let's get prepared for a huge amount of work!

Then you get to work with your team and define the following tasks to perform:   
1. You need to start your analysis using data from the past.  
2. You need to define a process that takes your daily data as an input and integrates it.  

You are in charge of the second part, so you are provided with a sample file that you will have to read daily. To complete you task, you need the following aggregates:
* One aggregate per store that adds up the rest of the values.
* One aggregate per item that adds up the rest of the values.

You can import the dataset `retail_sales` from Ironhack's database. 

## Your task
Therefore, your process will consist of the following steps:
1. Read the sample file that a daily process will save in your folder. 
2. Clean up the data.
3. Create the aggregates.
4. Write three tables in your local database: 
    - A table for the cleaned data.
    - A table for the aggregate per store.
    - A table for the aggregate per item.

## Instructions
* Read the csv you can find in Ironhack's database.
* Clean the data and create the aggregates as you consider.
* Create the tables in your local database.
* Populate them with your process.

In [1]:
# your code here

#### Prepare the workspace and import the data

In [2]:
## Import libraries
import pymysql
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

## Connect to DB
driver   = 'mysql+pymysql:'
user     = 'data-students' 
password = 'iR0nH@cK-D4T4B4S3'
ip       = '34.65.10.136' 
database = 'retail_sales' 


connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

## Import day data

raw_sales_original = pd.read_sql('SELECT * FROM raw_sales;', engine)
raw_sales          = pd.read_sql('SELECT * FROM raw_sales;', engine)


#### Check data headers (data relevance)

In [3]:
print(raw_sales.head(4))

        date  shop_id  item_id  item_price  item_cnt_day
0 2015-01-04       29     1469      1199.0           1.0
1 2015-01-04       28    21364       479.0           1.0
2 2015-01-04       28    21365       999.0           2.0
3 2015-01-04       28    22104       249.0           2.0


The data provided in the dataset is all relevant to the aggregates we want to create. Therefore I would not create a subset of
data to work with.

#### Check data structure (NaN values, types of data)

In [4]:
print('')
print(raw_sales.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4545 entries, 0 to 4544
Data columns (total 5 columns):
date            4545 non-null datetime64[ns]
shop_id         4545 non-null int64
item_id         4545 non-null int64
item_price      4545 non-null float64
item_cnt_day    4545 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 177.7 KB
None


**NaN values:** the current dataset contains no NaN. If there were rows with NaN values, and assuming that retrieving the
information again from the shop system would require time and effort that our client does not wish to invest, I will 
proceed as follows:

* For date: Since we are receiving daily values, I would assume that I can infer the value from the rest of the values in
the column.
* For shop_id: I won't discard the row as the info is partially valid (i.e: it can provide valuable info on the item aggregate table) Therefore I will replace NaN by 0 (assuming no shop_id = 0)

* For item_id: I won't discard the row as the info is partially valid (i.e: it can provide valuable info on the shop aggregate table) Therefore I will replace NaN by 0 (assuming no item_id = 0)

* For item_price: As the price can't be infered, I would discard the record. However, I will provide a warning message so measures can be taken if considered necessary.

* For item_cnt_day: As the sales per day can't be infered, I would discard the record. However, I will provide a warning message so measures can be taken if considered necessary.


In [5]:
def missing_val (raw_sales):
    """
    The function checks each column of the data and performs the following corrections:
    Date: inferes the date from the existing values (same date)
    Shop_id: replaces NaN for 0
    Item_id: replaces NaN for 0
    Input: the daily data extracted from the DB.
    Output: the data without NaN values
    """
    if raw_sales.isna().sum()['date'] != 0:
        existing_values_dates = raw_sales[~(raw_sales.date.isna())]
        raw_sales['date'] = raw_sales['date'].fillna(existing_values_dates.loc[0, 'date'])
    if raw_sales.isna().sum()['shop_id'] != 0:
        raw_sales['shop_id'] = raw_sales['shop_id'].fillna(0)
    if raw_sales.isna().sum()['item_id'] != 0:
        raw_sales['item_id'] = raw_sales['item_id'].fillna(0)
    if raw_sales.isna().sum()['item_price'] != 0:    
        records_drop_it_price = raw_sales.isna().sum()['item_price']
        percentage_drop_it_price = records_drop_it_price/len(raw_sales)
        print('Due to missing values in item_price column, you have', records_drop_it_price, 'record drop. That is a', percentage_drop_it_price, '% of the total dataframe rows.')
        raw_sales = raw_sales[~(raw_sales.item_price.isna())]
    if raw_sales.isna().sum()['item_cnt_day'] != 0:    
        records_drop_item_cnt_day = raw_sales.isna().sum()['item_cnt_day']
        percentage_drop_item_cnt_day = records_drop_item_cnt_day/len(raw_sales)
        print('Due to missing values in item_cnt_day column, you have', records_drop_it_price, 'record drop. That is a', percentage_drop_it_price, '% of the total dataframe rows.')
        raw_sales = raw_sales[~(raw_sales.item_cnt_day.isna())]
    return raw_sales
clean_raw_sales = missing_val (raw_sales)
print(clean_raw_sales)


           date  shop_id  item_id  item_price  item_cnt_day
0    2015-01-04       29     1469      1199.0           1.0
1    2015-01-04       28    21364       479.0           1.0
2    2015-01-04       28    21365       999.0           2.0
3    2015-01-04       28    22104       249.0           2.0
4    2015-01-04       28    22091       179.0           1.0
...         ...      ...      ...         ...           ...
4540 2015-01-04       15     4240      1299.0           1.0
4541 2015-01-04       14    21922        99.0           1.0
4542 2015-01-04       15     1969      3999.0           1.0
4543 2015-01-04       14    22091       179.0           1.0
4544 2015-01-04       15     1007      1199.0           1.0

[4545 rows x 5 columns]


**Type of data:** The defined types seem correct, with the exception of item_cnt_day, that could be stored as an integer assuming that sold quantities can't be decimals (that is, items can't be sold in pieces or parts) However, I would check with our client in that aspect before making any change, as changing the type will cause a loss of information. 

Nevertheless, if the data contained NaN at some point, the type of the columns will be changed (as an int column can't store a NaN). Therefore, I would create a function that sets the expected types in each column after the NaN correction is made.

In [6]:
dict_types = {'date': 'datetime64[ns]', 'shop_id': 'int64', 'item_id': 'int64', 'item_price': 'float64', 'item_cnt_day': 'float64'}
for col in raw_sales.columns:
    if raw_sales[col].dtypes != dict_types[col]:
        raw_sales = raw_sales.astype({col : dict_types[col]})    

#### Check data consistency (detect outliers)

In [7]:
print(raw_sales.describe())

           shop_id       item_id    item_price  item_cnt_day
count  4545.000000   4545.000000   4545.000000   4545.000000
mean     34.021122  11140.459406   1031.686121      1.103630
std      16.565517   6558.649572   2073.919990      0.536967
min       2.000000     30.000000      3.000000     -1.000000
25%      22.000000   4977.000000    249.000000      1.000000
50%      31.000000  11247.000000    479.000000      1.000000
75%      50.000000  16671.000000   1192.000000      1.000000
max      59.000000  22162.000000  27990.000000     10.000000


The quantitative description for **shop_id** and **item_id** does not provide usable information as the values in these columns are ids and not quantities.

For **item_price** we can see a great dispersion. The distribution is clearly left skewed and the max price seems to be very 
deviated from the mean. Even when calculating smaller percentiles, the higher percentile shows a great dispersion, indicating outlier(s) in the highest values of the series.


In [8]:
raw_sales = raw_sales_original
percentiles = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
raw_sales['bins'] = pd.qcut(raw_sales['item_price'], len(percentiles), labels = percentiles)
# raw_sales[ (raw_sales['bins'] == '10') ].sort_values(by='item_price', ascending = False)
print('The lowest price in the last percentile is ', raw_sales[ (raw_sales['bins'] == '10') ].item_price.min())
print('The highest price in the last percentile is ', raw_sales[ (raw_sales['bins'] == '10') ].item_price.max())

The lowest price in the last percentile is  2590.0
The highest price in the last percentile is  27990.0


In order to try and understand if that value may be an error, we order the prices and see if the following prices are as high as the first one. Since there seem to be several products with high prices, I would not discard those rows without talking to the client first.

In [9]:
prices_list = raw_sales.item_price
prices_list_unique = prices_list.drop_duplicates()
prices_list_unique.sort_values(ascending = False, inplace = True)
prices_list_unique.head(10)

904     27990.0
981     27392.0
400     26990.0
983     25392.0
1495    19990.0
663     14990.0
1298     8999.0
1152     6990.0
1123     6799.0
451      5890.0
Name: item_price, dtype: float64


For **item_cnt_day** we observe that the min value is -1. We should check with our client if that means that this item was returned (and therefore the value is correct) or if it is an error. In the case it was indeed an error, and we know for sure what the number stands for (i.e.: it stands for 0 but due to a system error it is stored as -1), we could replace it using the following formula:

In [10]:
# raw_sales['item_cnt_day'].replace(-1, 0, inplace = True)

However, in this case I would prefer not to change the value, as it makes more sense to me to consider -1 as a retunt than considering it a sale of 0 products (which won't actually be a sale).

#### Create agregated tables tables for shop view and item view

In [11]:
raw_sales.drop(columns="bins", inplace=True)

In [12]:
raw_sales['revenue'] = raw_sales['item_price']*raw_sales['item_cnt_day']
raw_sales.head(3)

,date,shop_id,item_id,item_price,item_cnt_day,revenue
0,2015-01-04,29,1469,1199.0,1.0,1199.0
1,2015-01-04,28,21364,479.0,1.0,479.0
2,2015-01-04,28,21365,999.0,2.0,1998.0


In [16]:
## Shop aggregated view:
shop_agg = raw_sales.groupby(['shop_id'], as_index = False).sum()
shop_final = shop_agg.drop(columns=['item_id','item_price', 'item_cnt_day'])

## Item aggregated view:
item_agg = raw_sales.groupby(['item_id'], as_index = False).sum()
item_final = item_agg.drop(columns=['item_id','item_price', 'item_cnt_day'])


print('The shop aggregate is: ')
print('')
print(shop_final)
print('')
print('The item aggregate is: ')
print('')
print(item_final)
print('')
print('The clean sales are: ')
print('')
print(raw_sales)



The shop aggregate is: 

    shop_id   revenue
0         2  103746.0
1         3   67443.0
2         4   29361.0
3         5   33138.0
4         6  138678.0
5         7   52371.0
6        10   22716.0
7        12  295173.0
8        14   57450.0
9        15  125139.0
10       16  121923.0
11       18   35787.0
12       19   51420.0
13       21  228999.0
14       22  150717.0
15       24   56955.0
16       25  288432.0
17       26  120462.0
18       27  172959.0
19       28  202512.0
20       29   85737.0
21       31  304692.0
22       34   12117.0
23       35   65769.0
24       37  220500.0
25       38   73482.0
26       39   34686.0
27       41   36840.0
28       42  330111.0
29       44  137445.0
30       45   82350.0
31       46   93903.0
32       47   80142.0
33       48   32745.0
34       49   35784.0
35       50  142053.0
36       51   10665.0
37       52   63531.0
38       53   50505.0
39       54  125343.0
40       55  170847.6
41       56   54906.0
42       57  226269.0
43     

In [14]:
# shop_final.to_sql('shop_view', con=engine, if_exists='append')
# item_final.to_sql('item_view', con=engine, if_exists='append')
# clean_sales.to_sql('clean_sales', con=engine, if_exists='append')